In [1]:
source("../config_sing.R")

── Attaching packages ──────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ─────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘pryr’


The following objects are masked from ‘package:purrr’:

    compose, partial



Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine




In [2]:
FD_RES

[1] "/home/mount/work/out/proj_combeffect"

In [43]:
get_trt = function(txt){
    if (str_detect(txt, "Input")){
        trt = "Input"
    }
    if (str_detect(txt, "_.*")){
        trt = str_extract(txt, "_.*")
        trt = paste0("TFX", trt)
    }
    if (str_detect(txt, "total")){
        trt = "total"
    }
    return(trt)
}

In [47]:
ctypes = c(col_integer(), col_character())
cnames = c("Size", "Fpath")

fdiry = file.path(FD_RES, "source")
fname = "library_size.txt"

dat_lib_size = read_tsv(
    file.path(fdiry, fname), 
    col_types = ctypes, 
    col_names = cnames)

dat_lib_size = dat_lib_size %>% 
    mutate(Sample=tools::file_path_sans_ext(basename(Fpath))) %>%
    dplyr::select(Size, Sample)

In [48]:
#SAMPLES = c(
#    paste0("Input", 1:5, "_20x"),
#    paste0("TFX",   2:5, "_DMSO"),
#    paste0("TFX",   2:5, "_Dex"))

SAMPLES = c(
    paste0("Input", 1:2, "_20x"),
    paste0("TFX",   2:3, "_DMSO"),
    paste0("TFX",   2:3, "_Dex"))
SAMPLES

[1] "Input1_20x" "Input2_20x" "TFX2_DMSO"  "TFX3_DMSO"  "TFX2_Dex"  
[6] "TFX3_Dex"

In [49]:
fdiry = file.path(FD_RES, "peak", "peak_count_cradle")
dir(fdiry)

[1] "Input1"     "Input1_20x" "Input2"     "Input2_20x" "Input3"    
 [6] "Input3_20x" "Input4"     "Input4_20x" "Input5"     "Input5_20x"
[11] "TFX2_Dex"   "TFX2_DMSO"  "TFX3_Dex"   "TFX3_DMSO"  "TFX4_Dex"  
[16] "TFX4_DMSO"  "TFX5_Dex"   "TFX5_DMSO"

In [50]:
fdiry = file.path(FD_RES, "peak", "peak_count_cradle", "Input1")
dir(fdiry)

[1] "chr1_count.bed.gz"      "chr1_intersect.bed.gz"  "chr10_count.bed.gz"    
 [4] "chr10_intersect.bed.gz" "chr11_count.bed.gz"     "chr11_intersect.bed.gz"
 [7] "chr12_count.bed.gz"     "chr12_intersect.bed.gz" "chr13_count.bed.gz"    
[10] "chr13_intersect.bed.gz" "chr14_count.bed.gz"     "chr14_intersect.bed.gz"
[13] "chr15_count.bed.gz"     "chr15_intersect.bed.gz" "chr16_count.bed.gz"    
[16] "chr16_intersect.bed.gz" "chr17_count.bed.gz"     "chr17_intersect.bed.gz"
[19] "chr18_count.bed.gz"     "chr18_intersect.bed.gz" "chr19_count.bed.gz"    
[22] "chr19_intersect.bed.gz" "chr2_count.bed.gz"      "chr2_intersect.bed.gz" 
[25] "chr20_count.bed.gz"     "chr20_intersect.bed.gz" "chr21_count.bed.gz"    
[28] "chr21_intersect.bed.gz" "chr22_count.bed.gz"     "chr22_intersect.bed.gz"
[31] "chr3_count.bed.gz"      "chr3_intersect.bed.gz"  "chr4_count.bed.gz"     
[34] "chr4_intersect.bed.gz"  "chr5_count.bed.gz"      "chr5_intersect.bed.gz" 
[37] "chr6_count.bed.gz"      "chr6_intersect.bed.gz"  "chr7_count.bed.gz"     
[40] "chr7_intersect.bed.gz"  "chr8_count.bed.gz"      "chr8_intersect.bed.gz" 
[43] "chr9_count.bed.gz"      "chr9_intersect.bed.gz"  "chrX_count.bed.gz"     
[46] "chrX_intersect.bed.gz"

## Test

In [51]:
ctypes = c(col_character(), col_integer(), col_integer(), col_character(), col_integer())
cnames = c("Chrom", "Start", "End", "Name", "Count")
lst_bed = lapply(SAMPLES, function(sam){
    fdiry = file.path(FD_RES, "peak", "peak_count_cradle", sam)
    fname = "chr22_count.bed.gz"
    fpath = file.path(fdiry, fname)
    grp   = str_remove(sam, "[0-9]")
    dat   = read_tsv(fpath, col_names = cnames, col_types = ctypes)
    dat   = dat %>% mutate(Sample = sam, Group = grp)
    return(dat)
})

In [56]:
dat = bind_rows(lst_bed)
dat = left_join(dat, dat_lib_size, by="Sample")
dat = dat %>% mutate(Norm_Count  = Count/Size)
head(dat)

Chrom,Start,End,Name,Count,Sample,Group,Size,Norm_Count
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
chr22,10961222,10962222,chr22:10961372-10962072:DMSO,1136,Input1_20x,Input_20x,371718546,3.056076e-06
chr22,11027024,11028024,chr22:11027074-11027974:Dex,410,Input1_20x,Input_20x,371718546,1.102985e-06
chr22,11053269,11054269,chr22:11053285-11054254:DMSO,840,Input1_20x,Input_20x,371718546,2.259774e-06
chr22,11053371,11054371,chr22:11053219-11054524:Dex,873,Input1_20x,Input_20x,371718546,2.348551e-06
chr22,11055724,11056724,chr22:11056124-11056324:Dex,1205,Input1_20x,Input_20x,371718546,3.241700e-06
chr22,11055981,11056981,chr22:11056374-11056589:Dex,1144,Input1_20x,Input_20x,371718546,3.077597e-06


In [57]:
dat = bind_rows(lst_bed)
dat = left_join(dat, dat_lib_size, by="Sample")
dat = dat %>% mutate(Norm_Count  = Count/Size)

dat = dat %>% 
    group_by(Chrom, Start, End, Name, Group) %>% 
    summarize(Value = mean(Norm_Count), .groups = "drop")
head(dat)

Chrom,Start,End,Name,Group,Value
<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
chr22,10961222,10962222,chr22:10961372-10962072:DMSO,Input_20x,2.993652e-06
chr22,10961222,10962222,chr22:10961372-10962072:DMSO,TFX_Dex,4.504651e-06
chr22,10961222,10962222,chr22:10961372-10962072:DMSO,TFX_DMSO,5.841114e-06
chr22,11027024,11028024,chr22:11027074-11027974:Dex,Input_20x,1.112425e-06
chr22,11027024,11028024,chr22:11027074-11027974:Dex,TFX_Dex,2.866254e-06
chr22,11027024,11028024,chr22:11027074-11027974:Dex,TFX_DMSO,2.850193e-06


In [27]:
dat %>% dplyr::filter(Name == "chr22:10961372-10962072:DMSO") %>% spread(Sample, Count)

Chrom,Start,End,Name,Input1_20x,Input2_20x,TFX2_Dex,TFX2_DMSO,TFX3_Dex,TFX3_DMSO
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr22,10961222,10962222,chr22:10961372-10962072:DMSO,1136,1019,158,249,146,161


In [62]:
tmp = dat %>% 
    spread(Group, Value) %>% 
    mutate(
        Log2_Score_DMSO = log2(TFX_DMSO) - log2(Input_20x),
        Log2_Score_Dex  = log2(TFX_Dex)  - log2(Input_20x)) %>%
    dplyr::select(Chrom, Start, End, Name, Log2_Score_DMSO, Log2_Score_Dex)
head(tmp)

Chrom,Start,End,Name,Log2_Score_DMSO,Log2_Score_Dex
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
chr22,10961222,10962222,chr22:10961372-10962072:DMSO,0.9643367,0.5895085
chr22,11027024,11028024,chr22:11027074-11027974:Dex,1.3573521,1.3654589
chr22,11053269,11054269,chr22:11053285-11054254:DMSO,1.1809666,3.7232373
chr22,11053371,11054371,chr22:11053219-11054524:Dex,1.1582129,3.6762012
chr22,11055724,11056724,chr22:11056124-11056324:Dex,0.4496085,1.0786770
chr22,11055981,11056981,chr22:11056374-11056589:Dex,0.3551987,1.0109531


In [31]:
head(tmp)

Chrom,Start,End,Name,Input1_20x,Input2_20x,TFX2_Dex,TFX2_DMSO,TFX3_Dex,TFX3_DMSO
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr22,10961222,10962222,chr22:10961372-10962072:DMSO,1136,1019,158,249,146,161
chr22,11027024,11028024,chr22:11027074-11027974:Dex,410,390,121,178,81,44
chr22,11053269,11054269,chr22:11053285-11054254:DMSO,840,689,1051,165,868,157
chr22,11053371,11054371,chr22:11053219-11054524:Dex,873,710,1056,173,868,157
chr22,11055724,11056724,chr22:11056124-11056324:Dex,1205,1065,333,183,158,119
chr22,11055981,11056981,chr22:11056374-11056589:Dex,1144,1068,322,152,140,118


## Run on each chromsome

In [72]:
SAMPLES = c(
    paste0("Input", 1:5, "_20x"),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))
SAMPLES

[1] "Input1_20x" "Input2_20x" "Input3_20x" "Input4_20x" "Input5_20x"
 [6] "TFX2_DMSO"  "TFX3_DMSO"  "TFX4_DMSO"  "TFX5_DMSO"  "TFX2_Dex"  
[11] "TFX3_Dex"   "TFX4_Dex"   "TFX5_Dex"

In [73]:
CHROMS = paste0("chr", c(1:22, "X"))
CHROMS

[1] "chr1"  "chr2"  "chr3"  "chr4"  "chr5"  "chr6"  "chr7"  "chr8"  "chr9" 
[10] "chr10" "chr11" "chr12" "chr13" "chr14" "chr15" "chr16" "chr17" "chr18"
[19] "chr19" "chr20" "chr21" "chr22" "chrX"

In [75]:
for (chrom in CHROMS){
    ### print START message
    cat(chrom, "\n")
    flush.console()
    
    ### import data
    ctypes = c(col_character(), col_integer(), col_integer(), col_character(), col_integer())
    cnames = c("Chrom", "Start", "End", "Name", "Count")
    lst_bed = lapply(SAMPLES, function(sam){
        fdiry = file.path(FD_RES, "peak", "peak_count_cradle", sam)
        fname = paste0(chrom, "_count.bed.gz")
        fpath = file.path(fdiry, fname)
        grp   = str_remove(sam, "[0-9]")
        dat   = read_tsv(fpath, col_names = cnames, col_types = ctypes)
        dat   = dat %>% mutate(Sample = sam, Group = grp)
        return(dat)
    })
    
    ### Merge and normalize the counts 
    dat = bind_rows(lst_bed)
    dat = left_join(dat, dat_lib_size, by="Sample")
    dat = dat %>% mutate(Norm_Count  = Count/Size)

    ### Calculate the mean counts across replicates
    dat = dat %>% 
        group_by(Chrom, Start, End, Name, Group) %>% 
        summarize(Value = mean(Norm_Count), .groups = "drop")
    
    ### Calculate the log2 scores for each treatment
    dat = dat %>% 
        spread(Group, Value) %>% 
        mutate(
            Log2_Score_DMSO = log2(TFX_DMSO) - log2(Input_20x),
            Log2_Score_Dex  = log2(TFX_Dex)  - log2(Input_20x)) %>%
        dplyr::select(
            Chrom, Start, End, Name, 
            Log2_Score_DMSO, 
            Log2_Score_Dex)
    
    ###
    fdiry = file.path(FD_RES, "peak", "peak_count_cradle", "results")
    fname = paste0("dat_score_", chrom, ".tsv")
    fpath = file.path(fdiry, fname)
    write_tsv(dat, fpath)
}

chr1 
chr2 
chr3 
chr4 
chr5 
chr6 
chr7 
chr8 
chr9 
chr10 
chr11 
chr12 
chr13 
chr14 
chr15 
chr16 
chr17 
chr18 
chr19 
chr20 
chr21 
chr22 
chrX 


In [67]:
head(dat)

Chrom,Start,End,Name,Log2_Score_DMSO,Log2_Score_Dex
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
chr22,10961222,10962222,chr22:10961372-10962072:DMSO,1.1163732,0.653345
chr22,11027024,11028024,chr22:11027074-11027974:Dex,0.8067387,1.531803
chr22,11053269,11054269,chr22:11053285-11054254:DMSO,1.4409862,3.990056
chr22,11053371,11054371,chr22:11053219-11054524:Dex,1.4094730,3.939340
chr22,11055724,11056724,chr22:11056124-11056324:Dex,0.6646528,1.322478
chr22,11055981,11056981,chr22:11056374-11056589:Dex,0.5091824,1.298918
